# Referenced Mosaic

# Setup

## Imports

In [ ]:
import os

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.utils import check_random_state

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('white')

In [ ]:
from night_horizons import utils, preprocess, mosaic, raster, pipelines

## Settings

In [ ]:
settings = {
    'referenced_image_dir': '/Users/Shared/data/nitelite/images/220513-FH135/referenced',
    'mosaic_filepath': '/Users/Shared/data/nitelite/mosaics/220513-FH135/referenced3.tiff',
    'img_log_fp': '/Users/Shared/data/nitelite/metadata/220513-FH135/image.log',
    'imu_log_fp': '/Users/Shared/data/nitelite/metadata/220513-FH135/PresIMULog.csv',
    'gps_log_fp': '/Users/Shared/data/nitelite/metadata/220513-FH135/GPSLog.csv',
    'overwrite': True,
    'random_state': 16849,
}

## Parse Settings

In [ ]:
random_state = check_random_state(settings['random_state'])

In [ ]:
referenced_fps = utils.discover_data(settings['referenced_image_dir'], ['tif', 'tiff'], pattern=r'Geo\s\d+_\d.tif')

# Extract/Transform/Load

## Fit the Sensor Georeferencing to Get Spatial Offset

In [ ]:
sensor_georeference_pipeline_X, sensor_georeference_pipeline_y = pipelines.GeoreferencePipelines.sensor_georeference()

In [ ]:
sensor_georeference_pipeline_X.set_params(nitelite__output_columns=['filepath', 'sensor_x', 'sensor_y'], sensor_georeference__passthrough=['filepath'])

In [ ]:
y = sensor_georeference_pipeline_y.fit_transform(referenced_fps)

In [ ]:
sensor_georeference_pipeline_X.fit(
    referenced_fps,
    y,
    nitelite__img_log_fp=settings['img_log_fp'],
    nitelite__imu_log_fp=settings['imu_log_fp'],
    nitelite__gps_log_fp=settings['gps_log_fp'],
)

In [ ]:
padding = sensor_georeference_pipeline_X.named_steps['sensor_georeference'].spatial_offset_

In [ ]:
X = sensor_georeference_pipeline_X.predict(referenced_fps)

## Create the Starting Image for the Mosaic

In [ ]:
import importlib
importlib.reload(mosaic)

In [ ]:
fit_row = X.iloc[0]
fit_df = pd.DataFrame([fit_row,])

In [ ]:
if os.path.isfile(settings['mosaic_filepath']) and settings['overwrite']:
    os.remove(settings['mosaic_filepath'])

In [ ]:
reffed_mosaic_pipeline = pipelines.MosaicPipelines.referenced_mosaic(settings['mosaic_filepath'])
reffed_mosaic_pipeline.set_params(mosaic__padding=padding, mosaic__exist_ok=True)

In [ ]:
less_reffed_mosaic_pipeline = mosaic.LessReferencedMosaic(filepath=settings['mosaic_filepath'], padding=padding)

In [ ]:
# Fit the less-reffed mosaic first because that will create the dataset
less_reffed_mosaic_pipeline.fit(X.iloc[1:][['filepath'] + preprocess.GEOTRANSFORM_COLS])

In [ ]:
# Fit the referenced mosaic to all the referenced images (with padding),
# s.t. the created mosaic is large enough
# reffed_mosaic_pipeline.fit(X['filepath'])

In [ ]:
reffed_mosaic_pipeline.named_steps['mosaic'].dataset_

In [ ]:
# Then create the referenced mosaic
reffed_mosaic_pipeline.fit_transform(fit_df['filepath'])

In [ ]:
# Get the data we'll use for the less-referenced mosaic,
# including the iteration order
X = X.iloc[1:]
X[['x_center', 'y_center']] - fit_df[['x_center', 'y_center']]

In [ ]:
# Finally, build off the referenced mosaic
less_reffed_mosaic_pipeline.predict(
    X.iloc[1:][['filepath'] + preprocess.GEOTRANSFORM_COLS],
)